In [1]:
import warnings
warnings.filterwarnings("ignore")
import pygeos
import rtree
import shapely
import geopandas as gpd
import pandas as pd
import pickle

In [2]:
def layers_dict_creation(base_path):
    '''Creates a dict of gdfs from the input .shp layers'''
    #base_path = 'C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Segunda iteración/layers_clipped'
    # Layers reading and slicing into 7 basic categories
    landuse_path = base_path + '/landuse_clipped.shp'
    landuse_gdf = gpd.read_file(landuse_path)
    # Landuse will give the parks surface
    parks_gdf = landuse_gdf[(landuse_gdf['fclass'] == 'grass') | (landuse_gdf['fclass'] == 'park')]
    parks_gdf = parks_gdf[['geometry']]
    # Buildings will give hospitals and school, the rest is considered residential
    buildings_path = base_path + '/buildings_clipped.shp'
    buildings_gdf = gpd.read_file(buildings_path)
    schools_gdf = buildings_gdf[(buildings_gdf['type'] == 'kindergarten') | (buildings_gdf['type'] == 'school') |\
                             (buildings_gdf['type'] == 'university')]
    hospitals_gdf = buildings_gdf[(buildings_gdf['type'] == 'hospital')]
    residential_gdf = buildings_gdf[(buildings_gdf['type'] != 'kindergarten') | (buildings_gdf['type'] != 'school') |\
                             (buildings_gdf['type'] != 'university') | (buildings_gdf['type'] != 'hospital')]
    # Roads are divided in primary, secondary or tertiary depending on their global network impact
    roads_path = base_path + '/roads_clipped.shp'
    roads_gdf = gpd.read_file(roads_path)
    primary_gdf = roads_gdf[(roads_gdf['fclass'] == 'primary') | (roads_gdf['fclass'] == 'primary_link')]
    secondary_gdf = roads_gdf[(roads_gdf['fclass'] == 'secondary') | (roads_gdf['fclass'] == 'service')]
    tertiary_gdf = roads_gdf[(roads_gdf['fclass'] == 'residential') | (roads_gdf['fclass'] == 'tertiary') \
                        | (roads_gdf['fclass'] == 'steps') | (roads_gdf['fclass'] == 'pedestrian') \
                        | (roads_gdf['fclass'] == 'living_street') | (roads_gdf['fclass'] == 'cycleway')]
    # The information is structured in a dict and outputed
    return_dict = {'residential_gdf': residential_gdf, 'schools_gdf': schools_gdf, 'parks_gdf': parks_gdf, \
                  'hospitals_gdf': hospitals_gdf, 'roads_1_gdf': primary_gdf, 'roads_2_gdf': secondary_gdf, \
                  'roads_3_gdf': tertiary_gdf}
    return return_dict
    
def buffers_dict_creation(distances_list):
    '''Creates a dict of gdfs from the buffers based in the traffic measurement stations'''
    base_path = 'C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Segunda iteración/layers_clipped'
    # A dict is created with every path depending on the filename, built from the distance list
    paths_dict = {}
    for distance in distances_list:
        path = base_path + '/buffer_' + str(distance) + 'm.shp'
        paths_dict[distance] = path
    buffers_dict = {}
    # Once the info is read, it is given in a dict as in the before function
    for key, path in paths_dict.items():
        buffers_dict[key] = gpd.read_file(path)
    return buffers_dict

def geoprocessing(buffers_dict, layers_dict, distances_list):
    '''Takes each layer, intersects them with the buffers and returns a pandas dataframe for each station'''
    return_dict = {}
    # We infer the stations from the buffer list (each buffer corresponds to a station)
    station_list = list(buffers_dict[100]['id'])
    # We loop through every station and then through every distance. The result will be structured in a dict of 
    # pandas dfs, one per station, giving the surface of intersection between buffer and particular land use
    # layer (in the case of polygons) and the distances of intersection between roads and buffers (in the case
    # of lines). Units are in SI.
    for station in station_list:
        for distance in distances_list:
            buffer = buffers_dict[distance]
            station_buffer = buffer[buffer['id'] == station]
            residential_intersection = gpd.overlay(layers_dict['residential_gdf'], station_buffer, how='intersection')
            schools_intersection = gpd.overlay(layers_dict['schools_gdf'], station_buffer, how='intersection')
            parks_intersection = gpd.overlay(layers_dict['parks_gdf'], station_buffer, how='intersection')
            hospitals_intersection = gpd.overlay(layers_dict['hospitals_gdf'], station_buffer, how='intersection')
            roads_1_intersection = gpd.overlay(layers_dict['roads_1_gdf'], station_buffer, how='intersection')
            roads_2_intersection = gpd.overlay(layers_dict['roads_2_gdf'], station_buffer, how='intersection')
            roads_3_intersection = gpd.overlay(layers_dict['roads_3_gdf'], station_buffer, how='intersection')
            residential_a = residential_intersection.area
            schools_a = schools_intersection.area
            parks_a = parks_intersection.area
            hospitals_a = hospitals_intersection.area
            roads_1_a = roads_1_intersection.length
            roads_2_a = roads_2_intersection.length
            roads_3_a = roads_3_intersection.length
            residential_a = residential_a.sum()
            schools_a = schools_a.sum()
            parks_a = parks_a.sum()
            hospitals_a = hospitals_a.sum()
            roads_1_a = roads_1_a.sum()
            roads_2_a = roads_2_a.sum()
            roads_3_a = roads_3_a.sum()
            keyslist = ['residential_a', 'schools_a', 'parks_a', 'hospitals_a', 'roads_1_a', 'roads_2_a', \
                       'roads_3_a']
            newkeyslist = []
            for key in keyslist:
                name = key + '_' + str(distance)
                newkeyslist.append(name)
            area_dict = {newkeyslist[0]: residential_a, newkeyslist[1]: schools_a, newkeyslist[2]: parks_a, \
                           newkeyslist[3]: hospitals_a, newkeyslist[4]: roads_1_a, newkeyslist[5]: roads_2_a, \
                           newkeyslist[6]: roads_3_a}
            if station not in return_dict.keys():
                station_df = pd.DataFrame(data=[area_dict])
                return_dict[station] = station_df
            elif station in return_dict.keys():
                for colname, value in area_dict.items():
                    return_dict[station][colname] = value
    return return_dict           

In [3]:
# Buffers are of a 25, 50, 100, 200, 500 and 1000m radius
# Geoprocess
distances = [25, 50, 100, 200, 500, 1000]
base_gdfs = layers_dict_creation('C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Segunda iteración/layers_clipped')
buffer_gdfs = buffers_dict_creation(distances)
geo_dict = geoprocessing(buffer_gdfs, base_gdfs, distances) 

In [4]:
# Exporting the dict

with open('C:/Users/Mikel/Desktop/Archivos/Estudios/Ciencia de Datos/TFM/Tercera iteración/geo_dict.pickle' \
          , 'wb') as handle:
    pickle.dump(geo_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)